<a href="https://colab.research.google.com/github/Tariqameenkhan/LangChain-RAG/blob/main/RAG-Method-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai  langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.6 MB/s eta 0:00:00


In [2]:
!pip install -qU pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.4 MB/s eta 0:00:00


In [3]:
import os
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')
userdata.get('PINECONE_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Ensure your VertexAI credentials are configured

from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")



In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings.embed_query("Hello world")



In [6]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec
api_key = os.environ.get('PINECONE_API_KEY')

pc = Pinecone(api_key= api_key, environment='us-east-1')  # Use your Pinecone environment

index_name = "helloworld"


pc.create_index(
      name=index_name,
      dimension=768, # Replace with your model dimensions
      metric="cosine", # Replace with your model metric
      spec=ServerlessSpec(
          cloud="aws",
          region="us-east-1"
      )
  )

index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents
loader = PyPDFLoader("name.pdf")  # Replace with your file
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(docs)

[Document(metadata={'source': 'name.pdf', 'page': 0}, page_content='My name is tariq. My name is ameen. My name is khan. My name is zain.')]


In [8]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
vectors=[]
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content)
    # Generate a unique ID for each document (e.g., UUID or hash)
    doc_id = str(hash(doc.page_content))
    metadata = {"text": doc.page_content}
    # Upsert the vector with a unique ID
    index.upsert(vectors=[(doc_id, vector,metadata)])


100%|██████████| 1/1 [00:00<00:00,  1.20it/s]


In [9]:
from langchain.vectorstores import Pinecone


retriever=  Pinecone(index=index, embedding=embeddings, text_key="text")


# Convert it into a retriever compatible with RetrievalQA
retriever = vector_store.as_retriever()

<ipython-input-9-b373e6fbbf23>:4: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  retriever=  Pinecone(index=index, embedding=embeddings, text_key="text")


In [10]:

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",  # Other options: "map_reduce", "refine"
    retriever=retriever
)


In [11]:
while True:
  user_input = input("Ask something: ")
  if user_input.lower() == "quit":
    break
  else:
    response = qa_chain.run(user_input)
    print(response)

Ask something: what is name


<ipython-input-11-d0d6668f0220>:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(user_input)


Based on the provided text, the names are Tariq, Ameen, Khan, and Zain.
Ask something: pakistan founder name
The provided text does not contain the name of the founder of Pakistan.
Ask something: explain ai
Based solely on the provided text, I cannot explain AI.  The text only gives a list of names and provides no information about artificial intelligence.  To explain AI, I would need access to information beyond that text.
Ask something: explain my
Based solely on the provided text, there is no information about "my".  The text only states the names of four people: Tariq, Ameen, Khan, and Zain.  Therefore, I cannot explain "my" in relation to this text.
Ask something: explain name
Based on the provided text, "name" refers to a person's identifier.  The text lists four names: Tariq, Ameen, Khan, and Zain.
Ask something: quit
